# 1단계: 탐색적 데이터 분석 (EDA)

## 📊 California Housing Dataset 분석

이 노트북에서는 California Housing Dataset을 로드하고 기초적인 탐색적 데이터 분석을 수행합니다.

### 학습 목표
1. California Housing Dataset 이해하기
2. 데이터의 기본 통계량 파악
3. 결측값 및 이상치 확인
4. 특성 간의 상관관계 분석
5. 데이터 시각화

In [ ]:
# 필요한 라이브러리 import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.datasets import fetch_california_housing
import sys
import os

# 상위 디렉토리의 src 폴더를 경로에 추가
sys.path.append('../src')

# 한글 폰트 설정
try:
    from font_setup import setup_korean_font
    setup_korean_font()
except:
    # 한글 폰트 설정이 실패하면 영어로 표시
    plt.rcParams['font.family'] = 'DejaVu Sans'
    print("⚠️ Korean font setup failed. Using English labels.")

# 시각화 설정
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# pandas 표시 옵션
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)

print("✅ Library import completed")

In [ ]:
# California Housing Dataset 로드
california_housing = fetch_california_housing()

# 데이터셋 정보 출력
print("📈 California Housing Dataset")
print(f"Number of features: {len(california_housing.feature_names)}")
print(f"Number of samples: {california_housing.data.shape[0]}")
print(f"Target variable: {california_housing.target_names}")

print("\n🏷️ Feature names:")
for i, feature in enumerate(california_housing.feature_names):
    print(f"{i+1:2d}. {feature}")

In [ ]:
# 데이터프레임 생성
df = pd.DataFrame(
    california_housing.data, 
    columns=california_housing.feature_names
)
df['MedHouseVal'] = california_housing.target

print("📊 DataFrame basic information")
print(f"Data shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# 처음 5행 표시
print("\n🔍 First 5 rows:")
df.head()

In [ ]:
# 데이터셋 설명 출력
print("📝 Dataset description:")
print(california_housing.DESCR[:1500])  # 처음 1500자만 표시

In [ ]:
# 기본 통계량 확인
print("📊 Basic statistics")
df.describe()

In [ ]:
# 데이터 타입 및 결측값 확인
print("🔍 Data types and missing values")
info_df = pd.DataFrame({
    'Data Type': df.dtypes,
    'Non-Null Count': df.count(),
    'Null Count': df.isnull().sum(),
    'Null Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})
print(info_df)

In [ ]:
# 상관관계 매트릭스
plt.figure(figsize=(12, 10))
correlation_matrix = df.corr()

# 히트맵 생성
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(
    correlation_matrix, 
    mask=mask,
    annot=True, 
    cmap='RdBu_r', 
    center=0,
    fmt='.3f',
    square=True,
    cbar_kws={"shrink": .8}
)
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# 타겟과 높은 상관관계를 가진 특성 확인
target_corr = correlation_matrix['MedHouseVal'].abs().sort_values(ascending=False)
print("\n🎯 Correlation with target (House Price):")
for feature, corr in target_corr.items():
    if feature != 'MedHouseVal':
        print(f"{feature:15s}: {corr:.3f}")

In [ ]:
# 분포 시각화 - 히스토그램
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.flatten()

for i, column in enumerate(df.columns):
    axes[i].hist(df[column], bins=50, alpha=0.7, edgecolor='black', linewidth=0.5)
    axes[i].set_title(f'{column} Distribution', fontweight='bold')
    axes[i].set_xlabel(column)
    axes[i].set_ylabel('Frequency')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('California Housing Dataset - Feature Distributions', y=1.02, fontsize=16, fontweight='bold')
plt.show()

In [ ]:
# 박스플롯으로 이상치 확인
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.flatten()

for i, column in enumerate(df.columns):
    bp = axes[i].boxplot(df[column], patch_artist=True)
    bp['boxes'][0].set_facecolor('lightblue')
    bp['boxes'][0].set_alpha(0.7)
    axes[i].set_title(f'{column} Boxplot', fontweight='bold')
    axes[i].set_ylabel(column)
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Outlier Detection - Boxplots', y=1.02, fontsize=16, fontweight='bold')
plt.show()

In [ ]:
# 지리적 데이터 시각화 (위도, 경도 활용)
plt.figure(figsize=(12, 8))
scatter = plt.scatter(
    df['Longitude'], 
    df['Latitude'], 
    c=df['MedHouseVal'], 
    cmap='viridis', 
    alpha=0.6,
    s=20
)
plt.colorbar(scatter, label='House Price (unit: $100K)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('California House Price Geographic Distribution', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 인구 밀도와 주택 가격의 관계
# 인구 밀도 = Population / (평균 가구당 인원 수 * 총 가구 수)
df['PopulationDensity'] = df['Population'] / (df['AveOccup'] * df['HouseAge'])

plt.figure(figsize=(12, 5))

# 서브플롯 1: 인구 밀도 vs 주택 가격
plt.subplot(1, 2, 1)
plt.scatter(df['PopulationDensity'], df['MedHouseVal'], alpha=0.5, s=10)
plt.xlabel('Population Density')
plt.ylabel('House Price (unit: $100K)')
plt.title('Population Density vs House Price')
plt.grid(True, alpha=0.3)

# 서브플롯 2: 평균 소득 vs 주택 가격
plt.subplot(1, 2, 2)
plt.scatter(df['MedInc'], df['MedHouseVal'], alpha=0.5, s=10, color='orange')
plt.xlabel('Median Income (unit: $10K)')
plt.ylabel('House Price (unit: $100K)')
plt.title('Median Income vs House Price')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 상관관계 확인
print(f"Population density vs House price correlation: {df['PopulationDensity'].corr(df['MedHouseVal']):.3f}")
print(f"Median income vs House price correlation: {df['MedInc'].corr(df['MedHouseVal']):.3f}")

In [ ]:
# 주요 통계 요약
print("📋 Key Findings Summary")
print("=" * 50)
print(f"• Total data points: {len(df):,}")
print(f"• Number of features: {len(df.columns)-1}")
print(f"• Missing values: {'None' if df.isnull().sum().sum() == 0 else 'Present'}")
print(f"• House price range: ${df['MedHouseVal'].min():.1f}K ~ ${df['MedHouseVal'].max():.1f}K")
print(f"• Average house price: ${df['MedHouseVal'].mean():.1f}K")
print(f"• Median income range: ${df['MedInc'].min():.1f}K ~ ${df['MedInc'].max():.1f}K")

print("\n🔍 Important features (based on target correlation):")
important_features = target_corr.head(4).index.tolist()
important_features.remove('MedHouseVal')
for i, feature in enumerate(important_features, 1):
    corr_value = target_corr[feature]
    print(f"{i}. {feature}: {corr_value:.3f}")

print("\n✅ EDA completed! Next step: Compare normalization effects.")

In [ ]:
# 데이터 저장 (다음 단계에서 사용)
df.to_csv('../data/california_housing_processed.csv', index=False)
print("💾 Processed data saved to 'data/california_housing_processed.csv'")